In [1]:
from facenet_pytorch import MTCNN
import cv2
from PIL import ImageDraw
import numpy as np
import torch
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import time
import glob
from moviepy.editor import *
from PIL import Image as Img
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
# Create face detector
mtcnn = MTCNN(margin=20, keep_all=True, post_process=False, device=device)
print(device)

cuda


In [3]:
# Load a video
v_cap = cv2.VideoCapture('./input.mp4')
v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Loop through video
batch_size = 32
frames = []
boxes = []
landmarks = []
all_frames = []
probs = []
for _ in tqdm(range(v_len)):
    
    # Load frame
    success, frame = v_cap.read()
    if not success:
        continue
        
    # Add to batch, resizing for speed
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    all_frames.append(frame)
    frame = Img.fromarray(frame)
    frames.append(frame)
    
    
    # When batch is full, detect faces and reset batch list
    if len(frames) >= batch_size:
        batch_boxes, p, batch_landmarks = mtcnn.detect(frames, landmarks=True)
        boxes.extend(batch_boxes)
        probs.extend(p)
        frames = []

  0%|          | 0/1294 [00:00<?, ?it/s]

In [4]:
# frame_ = all_frames[0].copy()
# im = Img.fromarray(frame_)
# draw = ImageDraw.Draw(im)
# for box in boxes[0]:
#     draw.rectangle(box.tolist(), outline=(255, 0, 0), width=6)
# im.show()

In [5]:
print(len(boxes))

1280


In [6]:
fps = int(v_cap.get(cv2.CAP_PROP_FPS))
fps

25

In [7]:
# Draw Faces
frames_with_faces = []

for ix in tqdm(range(len(all_frames))):
    frame_draw = all_frames[ix].copy()
    frame_draw = Img.fromarray(frame_draw)
    draw = ImageDraw.Draw(frame_draw)
    try:
        for box in boxes[ix]:
            draw.rectangle(box.tolist(), outline=(255, 0, 0), width=6)
    except TypeError as e:
        pass
    except IndexError as e:
        pass
    frames_with_faces.append(frame_draw)
        

  0%|          | 0/1294 [00:00<?, ?it/s]

In [8]:
print(len(frames_with_faces))

1294


In [11]:
writer = cv2.VideoWriter('test_temp.mp4', cv2.VideoWriter_fourcc(*'MJPG'), fps, (1280,720))
for i in tqdm(range(len(frames_with_faces))):
    img = cv2.cvtColor(np.array(frames_with_faces[i]).astype('uint8'),cv2.COLOR_BGR2RGB)
    writer.write(img)
writer.release()

  0%|          | 0/1294 [00:00<?, ?it/s]

In [12]:
videoOUT = VideoFileClip('test_temp.mp4')
audio = VideoFileClip('input.mp4').audio
videoOUT = videoOUT.set_audio(audio)
videoOUT.write_videofile('out.mp4', fps=fps)

chunk:   0%|          | 0/1142 [00:00<?, ?it/s, now=None]

Moviepy - Building video out.mp4.
MoviePy - Writing audio in outTEMP_MPY_wvf_snd.mp3


t:   0%|          | 0/1294 [00:00<?, ?it/s, now=None]                 

MoviePy - Done.
Moviepy - Writing video out.mp4



Moviepy - Done !
Moviepy - video ready out.mp4
